In [1]:
import os
import pandas as pd
import dask.dataframe as dd
import random
import json
import emoji
import dask
import json
import dask.dataframe as dd
import dask.bag as db
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
random.seed(493)
client

ModuleNotFoundError: No module named 'dask'

In [7]:
def process_raw_data(path, name, client=client):
    if not os.path.exists(f"../data/{name}.pkl"):
        print(client, client.dashboard_link)
        b = db.read_text(f'{path}/x**').map(json.loads)

        flatten = lambda rec: {
                    'full_text': rec['full_text'],
                    'created_at': rec['created_at'],
                    'id': rec['id'],
                    'lon': rec['coordinates']['coordinates'][0],
                    'lat': rec['coordinates']['coordinates'][1]}

        tweets_df = b.filter(lambda record: record['coordinates'] is not None)
        tweets_df = tweets_df.map(flatten).to_dataframe().compute()
        print(f"saving ../data/{name}.pkl")
        tweets_df.drop_duplicates(subset=['id']).to_pickle(f"../data/{name}.pkl")
    else:
        tweets_df = pd.read_pickle(f"../data/{name}.pkl").drop_duplicates(subset=['id'])
    return tweets_df


all_tweets_df = process_raw_data("../data/big_data/harvey_data", 'hbig')
all_tweets_df.head()

,full_text,created_at,id,lon,lat
0,#Harvey continues westward over the eastern Ca...,Sat Aug 19 02:40:43 +0000 2017,898736456918749200,-64.1000,13.7000
1,⒍ Inapa\n⒎ España\n⒏ Yunesky Maya\n⒐ Samaná\n⒑...,Sat Aug 19 12:20:09 +0000 2017,898882274808053800,-69.9405,18.4867
2,⒍ #LLWS\n⒎ España\n⒏ #Harvey\n⒐ Cambrils\n⒑ #P...,Sat Aug 19 12:40:09 +0000 2017,898887306509144000,-69.9405,18.4867
3,⒍ Inapa\n⒎ España\n⒏ #Harvey\n⒐ Cambrils\n⒑ #P...,Sat Aug 19 13:00:09 +0000 2017,898892341297139700,-69.9405,18.4867
4,#Harvey still a tropical storm but less organi...,Sat Aug 19 15:00:49 +0000 2017,898922707991113700,-68.1000,13.9000


In [8]:
def preprocess(text):
    import preprocessor as p
    import html
    from multiprocessing.dummy import Pool as ThreadPool    
    p.set_options(p.OPT.ESCAPE_CHAR, p.OPT.URL)
    pool = ThreadPool()
    results = pool.map(lambda d : p.clean(html.unescape(d)), text)
    pool.close()
    pool.join()
    return results

all_tweets_df = all_tweets_df[all_tweets_df['full_text'].str.len() != 0]
all_tweets_df['full_text'] = preprocess(list(all_tweets_df['full_text'].values))
all_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5609 entries, 0 to 365
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   full_text   5609 non-null   object 
 1   created_at  5609 non-null   object 
 2   id          5609 non-null   int64  
 3   lon         5609 non-null   float64
 4   lat         5609 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 262.9+ KB


In [9]:
all_tweets_df['full_text'].sample(5).values

array(['#HurricaneHarvey #Accuweather #KendisGibson #DianeMacedo #wnnfans #atmfans #7Steps2Quit @…',
       "#HurricaneHarvey formal attire - ✔ # Harvey2017 be safe all y'all #Texas.",
       "#harvey wouldn't want to be this guy running west in this mess! @ The Grand Marlin of Pensacola…",
       '@houstonspca has dog food available #hurricaneharvey #houstonstrong @ Houston, Texas',
       'Keep your eyes on the skies. Safety first. Play it smart. #hurricaneharvey @ Austin, Texas'],
      dtype=object)

In [10]:
def get_inexact_location(latlong):
    import reverse_geocoder as rg
    results = rg.search(latlong)
    return pd.DataFrame(results)
locs = get_inexact_location(list(zip(all_tweets_df['lat'], all_tweets_df['lon'])))
locs

Loading formatted geocoded file...


,lat,lon,name,admin1,admin2,cc
0,11.10783,-63.85537,La Plaza Paraguachi,Nueva Esparta,Municipio Antolin del Campo,VE
1,18.5,-69.93333,La Agustina,Nacional,,DO
2,18.5,-69.93333,La Agustina,Nacional,,DO
3,18.5,-69.93333,La Agustina,Nacional,,DO
4,12.23333,-68.33333,Dorp Rincon,Bonaire,,BQ
...,...,...,...,...,...,...
5604,40.71427,-74.00597,New York City,New York,,US
5605,30.18022,-95.45577,Shenandoah,Texas,Montgomery County,US
5606,30.18022,-95.45577,Shenandoah,Texas,Montgomery County,US
5607,29.76328,-95.36327,Houston,Texas,Harris County,US


In [12]:
all_tweets_df = all_tweets_df.merge(locs, left_index=True, right_index=True).reset_index(drop=True)
all_tweets_df.to_pickle("../data/hcleaned.pkl", protocol=4)

In [ ]:
# scp -r ./data/hcleaned.pkl atharva2@linux.ews.illinois.edu:~/LING_506_MAC/data/
# ssh onto linux.ews.illinois.edu
# big_proc.ipynb